In [1]:
from helpers import *
pl = PlParse(r"C:\Users\DELL\Downloads\PWSAutomate\multi\multisingle\pl.pdf")
print(pl.extracted_data)

TAIWAN Panalpina
{'Invoice Number': None, 'Log No': None, 'destination': 'TAIWAN', 'Document Date': None, 'Invoice Date': None, 'notes': 'Adidas Branded Garments. Export from Export Facilitation Scheme (EFS) 134740069 Unisex Infants 78% Cotton 22% Recycled Polyester Knit Pullover Dyed', 'final_table': {'Quantity': 507.0, 'Cartons': 18.0, 'Gross Weight': 98.648, 'Net Weight': 80.648, 'Net Net Weight': 72.536}, 'consignee': ' adidas Taiwan Limited', 'address': ' 13/F, No.2, Sec.3 Min Quan East Road Taipei, 104'}


In [1]:
# selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchWindowException,TimeoutException
from selenium.webdriver.support.ui import Select
import time
import math
import argparse
from helpers import *
import os
from datetime import datetime
from selenium.webdriver.chrome.service import Service as ChromeService
from chromedriver_py import binary_path
def setup_driver():
    svc = webdriver.ChromeService(executable_path=binary_path)
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-gpu')
    # chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-software-rasterizer')
    driver = webdriver.Chrome(options=chrome_options,service=svc)
    
    return driver

def exemptions(driver: webdriver.Chrome):
    # Add SROs/Exemptions
    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl02_lnkBtnAdd")
    select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl02_ddlExemption",option_text = r"SRO803(I)/2006-1-0-0-01/07/2006 ( 0 )")
    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl02_lnkBtnAdd")
    
    # Add another
    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl03_lnkBtnAdd")
    select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl03_ddlExemption",option_text = r"SRO575(I)/02-1-0-0-31/08/2002 ( 0 )")
    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl03_lnkBtnAdd")
    

def assessment_purpose(driver: webdriver.Chrome):
    
    # Add Assessment Purpose
    text = extract_text(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtQuantity")
    print(f"Extracted Text is {text}")
    write_text(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtQuantity_Statistical_Purpose",text=text)
    # write_text(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtQuantity_International_Traded",text=text)
    # Now Click 
    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_btnCalcExportValue")
    time.sleep(3)
    # write_text(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtActualWeight",text="1002.0000")

def populate_username_passowrd_in_login_form(driver,transaction_id,url, username, password):
    status = False
    error = ''

    try:
        driver.get(url)

        username_input = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder='Enter Username']"))
        )
        password_input = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder='Enter Password']"))
        )

        username_input.send_keys(username)
        password_input.send_keys(password)
        sign_in_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Login')]")
        sign_in_button.click()
        status = True
    except TimeoutException:
        error = f"For transaction_id { transaction_id } Login Page did not Appear."
    except Exception as e:
        error = f"For transaction_id { transaction_id } Error Occured => {str(e)}"
    return status, error
def select_types(driver,transaction_id):
    Create_Export_GD = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "//a[contains(., 'Create Export GD')]"))
    )
    print(f"Found Create_Export_GD.")
    # Perform an action on the input element (e.g., click)
    Create_Export_GD.click()
    select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_GdSelectionExport1_ddlConsignmentType",option_text = "Export Facilitation Scheme")
    select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_GdSelectionExport1_ddlDeclarationType",option_text = "Export Facilitation Scheme")

    Create_button = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "(//input[@value='Create'])[2]"))
    )
    print(f"Found Create_button.")
    # Perform an action on the input element (e.g., click)
    Create_button.click()
    print(f"Clicked Create_button.")

def select_saved(driver,transaction_id):
    saved_GD = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_lnkExportSaved"))
    )
    print(f"Found saved_GD.")
    saved_GD.click()
    GD = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_dgSaved_ctl02_lbTrackingId"))
    )
    print(f"Found GD.")
    GD.click()
    GD = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_btnSaveTop"))
    )
    print("GD ready")

def select_GDS(driver,transaction_id):
    status = False
    error = ''
    print(f"For transaction_id { transaction_id } Selecting GDS...")
    try:
        traders_button = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='List of Traders'])[2]"))
        )
        print(f"Found traders_button.")
        traders_button.click()
        iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
        driver.switch_to.frame(iframe)
        print(f"iframe Switched.")
        time.sleep(3)
        company_element = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "//td[text()='STYLE TEXTILE (PRIVATE) LTD']/preceding-sibling::td[1]//input[@type='image']"))
        )
        print(f"Found company_element.")
        # Perform an action on the input element (e.g., click)
        company_element.click()
        driver.switch_to.default_content()
        time.sleep(3)
        print(f"Switched to default_content.")
        goods_declaration = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='Goods Declaration'])[2]"))
        )
        print(f"Found goods_declaration.")
        goods_declaration.click()
        driver.switch_to.frame(iframe)
        select_types(driver,transaction_id)
        # select_saved(driver,transaction_id)
        status = True
    except Exception as e:
        error = f"For transaction_id { transaction_id } Error Occured => {str(e)}"
        print(e)
    return status, error

def fill_form(driver : webdriver.Chrome,transaction_id,data={}):
    status = False
    error = ''
    pl_data = data.get("pl_data")
    fty_data = data.get("fty_data")
    print(f"For transaction_id { transaction_id } Filling the Form...")
    try:
        # Basic Info
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlCollectorate",option_text = "Port Qasim (exports), karachi")
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlModeOfTransport",option_text = "Inland water Transport")
        
        # Consignor & Consignee Information (Hide)
        write_text(driver, "ctl00_ContentPlaceHolder2_ConsigneeInfoUc1_txtConsignorName", pl_data.get("consignee"))
        write_text(driver, "ctl00_ContentPlaceHolder2_ConsigneeInfoUc1_txtConsignorAddress", pl_data.get("address"))
        
        # GD Information
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlConsignmentMode",option_text = "Part Shipment")
        # SET the BL/AWB No (Invoice No Last Six digits)
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtBlNo", fty_data.get("extracted_data")[0].get('invoice_number'))
        #  SET the BL/AWB Date (Invoice Date)
        # Parse the date string into a datetime object
        date_object = datetime.strptime(fty_data.get("extracted_data")[0].get('invoice_date'), "%Y-%m-%d")
        write_date(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtBlDate_txtDate", date_object.strftime("%d/%m/%Y"))
        #  Select the Port of Shipment dropdown
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlPortOfShipment",option_text = "Port Qasim (exports), karachi")
        #  Select the Destination Country
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlDestinationCountry",option_text = "United States")
        #  Select the Destination Port
        select_dropdown_by_value(driver=driver,id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlPortOfDischarge",value = "41846")
        # Select the Shipping Line
        select_dropdown_by_value(driver=driver,id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlShippingline",value = "66")
        # SET Place of Delivery
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtPlaceofDelivery", "Long Beach, USA")
        # SET Net Weight (MT)
        net_weight = data.get("final_table").get("Net Weight") / 1000
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtNetWeight", net_weight)
        # SET Gross Weight
        gross_weight = data.get("final_table").get("Gross Weight") / 1000
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtGrossWeight", gross_weight)
        # SET Marks
        marks = f"""AS PER SHIPPER \n INVOICE NO.\n{data.get("final_table").get('invoices')}"""
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtMarks", marks)

        # financials_info
        # Select the PAYMENT TERM dropdown
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_FinancialsUc1_ddlPaymentTerms",option_text = "Without LC")
        # Select the Bank dropdown
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_FinancialsUc1_ddlBank",option_text = "MCB BANK LIMITED")
        # Select the Delivery Term dropdown
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_FinancialsUc1_ddlDeliveryTerm",option_text = "Free On Board (FOB)")
        # Select the Currency dropdown
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_FinancialsUc1_ddlCurrency",option_text = "United States-US $")
        # SET FOB Value
        write_text(driver, "ctl00_ContentPlaceHolder2_FinancialsUc1_txtFobValue", data.get("final_table").get("PO Net Amount"))

        # supporting_info_fill

        # Select the Shed/Location Code dropdown
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_SupportingInfoUc1_ddlExaminerGroup",option_text = "M/S. D.P. World Off-Dock Terminal")
        # Select the ddlTerminal dropdown
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_SupportingInfoUc1_ddlTerminal",option_text = "Qasim International Container Terminal")

        # check_disclaimer
        checkbox(driver,id="ctl00_ContentPlaceHolder2_ExportPolicyDisclaimerUc1_chkExportPolicy")
        checkbox(driver,id="ctl00_ContentPlaceHolder2_ExportPolicyDisclaimerUc1_chkIsSroDisclaimer")

        # Click the Save button
        click_button(driver=driver,id="ctl00_ContentPlaceHolder2_btnSaveBottom")

        print(f"GD Filling Success")
    except Exception as e:
        error = f"For transaction_id { transaction_id } Error Occured => {str(e)}"
        print(e)
    return status, error

def upload_document(driver : webdriver.Chrome,filepath:os.path.join):
    UploadButton = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "//a[@id='ctl00_ContentPlaceHolder2_UploadDocumentUc1_lnkUploadDocument' and text()='Upload Document']"))
    )
    print(f"Found Upload Button.")
    UploadButton.click()
    fileButton = WebDriverWait(driver, 1000).until(
        EC.presence_of_element_located((By.XPATH, "//table[.//span[text()='Attached Document']]//input[@id='ctl00_ContentPlaceHolder2_GdExportUploadDocUc1_GdImportDocUpload_fuDoc']"))
    )
    print("fileButton ready")
    fileButton.send_keys(filepath)
    if 'fty' in str(filepath).lower():
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_GdExportUploadDocUc1_cmbDocumentType",option_text = "Invoice")
    else:
        select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_GdExportUploadDocUc1_cmbDocumentType",option_text = "Packing List")
    UploadDocButton = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "//table[.//span[text()='Attached Document']]//input[@id='ctl00_ContentPlaceHolder2_GdExportUploadDocUc1_btnUpload' and @value='Upload']"))
    )
    print(f"Found Upload DOC Button.")
    UploadDocButton.click()
    
    WebDriverWait(driver, 500).until(
        EC.presence_of_element_located((By.XPATH, "//table[.//span[text()='Customs Office']]//select[@id='ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlCollectorate']"))
    )
    print("GD ready")

def upload_documents(driver : webdriver.Chrome,pdf_path):
    
    for file in pdf_path:
        print(f"Uploading file: {file}")
        upload_document(driver, file)
def fill_container_info(driver : webdriver.Chrome,data={}):
    # Now Click on Container Info

    click_button(driver=driver,id="//table[.//span[text()='Containers Information']]//a[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_lnkBtnAdd']",by=By.XPATH)

    write_text(driver=driver,id="//table[.//span[text()='Containers Information']]//input[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_txtContainerNo']",text="STYLE123456",by=By.XPATH)
    quantity = str(data.get('Quantity'))
    write_text(driver=driver,id="//table[.//span[text()='Containers Information']]//input[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_txtQuantity']",text=quantity,by=By.XPATH)
    cartons = str(data.get('Carton'))
    write_text(driver=driver,id="//table[.//span[text()='Containers Information']]//input[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_txtNoOfPackages']",text=cartons,by=By.XPATH)
    select_dropdown(driver=driver,id="//table[.//span[text()='Containers Information']]//select[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_ddlPackageType']",option_text="CARTONS",by=By.XPATH)
    click_button(driver=driver,id="//table[.//span[text()='Containers Information']]//a[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_lnkBtnAdd']",by=By.XPATH)


def add_item(driver : webdriver.Chrome,transaction_id,data={}):
    print(f"For transaction_id { transaction_id } Adding Item...")
    click_button(driver=driver,id="//a[@id='ctl00_ContentPlaceHolder2_ItemInfoUc1_lnkItems' and text()='Add Items']",by=By.XPATH)
    print("Clicked Add Items")
    write_text(driver, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtHsCode", data.get('hs_code'))
    write_text(driver, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtDeclaredDescription", data.get("description"))
    select_dropdown_by_value(driver,'ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_ddlOrigion','586')
    unit_value = (data.get('PO Net Amount')) / (data.get('Quantity'))
    write_text(driver, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtUnitValue", unit_value)
    write_text(driver, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtActualWeight", data.get('Quantity'))
    # Select Actual Unit now
    select_dropdown(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_ddlActualWeightUnit",option_text = "NO")
    print("filling the exemptions")
    exemptions(driver)
    # Check	BLEACHED BLENDED GARMENTS, WEARING APPAREL (ALL BLENDS OF POLYESTER STAPLE FIBRE AND COTTON FIBRE).
    checkbox(driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgSro_ctl03_chkSro")
    # Now Handel Quantity (for Assessment Purpose)
    time.sleep(5)
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "//table[.//span[text()='HS Code']]//input[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtHsCode']"))
    )
    print(f"Now Executing assessment_purpose")
    
    fill_container_info(driver=driver,data=data)
    assessment_purpose(driver)
    # Now Click on Save Button
    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_btnSaveBottom")
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_BasicInfoUc1_pnlTitle"))
    )
    print("HScode Added")
    return data.get('hs_code')

def process_gd_number_pop_up_492(driver : webdriver.Chrome,data):
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_btnGDLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",data.get('B/E No'),pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(5)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookup_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    # Locate the table
    table = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_dgItems"))
        )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Iterate through the rows, skipping the header
    for row in rows[1:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            try:
                unit_value = float(cells[4].text)
                puv = float(data.get('PER UNIT VALUE'))
                if math.isclose(unit_value, puv, rel_tol=0.1):
                    select_link = cells[0].find_element(By.TAG_NAME, "a")
                    if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                        select_link.click()
                    else:
                        return None
                    time.sleep(2)
                    break
            except ValueError:
                print(f"Skipping row due to non-numeric value: {cells[4].text}")
    else:
        # select 1st row
        row = rows[1]
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            select_link = cells[0].find_element(By.TAG_NAME, "a")
            print(f"No matching row found in the table for PER UNIT VALUE {data.get('PER UNIT VALUE')} and {data.get('B/E No')} Selecting 1st row")
            if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                select_link.click()
            else:
                print("Select Link is not enabled")
                return None
        else:
            return None
    try:
        Quantity = float(extract_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity"))
    except :
        Quantity = 0.0
    try:
        CONSUMED = float(data.get('NOW CONSUMED'))
    except :
        CONSUMED = 0.0
    if CONSUMED < Quantity:
        write_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity",data.get('Now Consume'),pop_up=True)
    return True

def process_gd_number_pop_up_957(driver : webdriver.Chrome,data,is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_btnGDLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",data.get(be_no),pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(5)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookup_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    # Locate the table
    table = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_dgItems"))
        )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Iterate through the rows, skipping the header
    for row in rows[1:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            print("The celss text is : ",cells[4].text)
            try:
                unit_value = float(cells[4].text)
                puv = float(data.get('PER UNIT VALUE'))
                if math.isclose(unit_value, puv, rel_tol=0.1):
                    select_link = cells[0].find_element(By.TAG_NAME, "a")
                    if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                        select_link.click()
                    else:
                        return None
                    time.sleep(2)
                    break
            except ValueError:
                print(f"Skipping row due to non-numeric value: {cells[4].text}")
    else:
        print('In rows')
        # select 1st row
        row = rows[1]
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            select_link = cells[0].find_element(By.TAG_NAME, "a")
            print(f"No matching row found in the table for PER UNIT VALUE {data.get('PER UNIT VALUE')} and {data.get(be_no)} Selecting 1st row")
            if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                select_link.click()
                time.sleep(5)
            else:
                print("Select Link is not enabled")
                return None
        else:
            return None
    try:
        Quantity = float(extract_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity"))
    except :
        Quantity = 0.0
    try:
        CONSUMED = float(data.get('NOW CONSUMED'))
    except :
        CONSUMED = 0.0
    if CONSUMED < Quantity:
        write_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity",data.get('NOW CONSUMED'),pop_up=True)
    hs_code = extract_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtHsCode")
    return hs_code

def process_analysis_number_pop_up_957(driver : webdriver.Chrome,analysis_number,hs_code):
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_btnAnalysisLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",analysis_number,pop_up=True)
    write_text(driver, "txtInputHSCode",hs_code,pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(3)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookupExport_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    time.sleep(2)
    wait_for_page_load(driver)

def add_excel_data_492(driver: webdriver.Chrome, data):
    # Wait until the image is present
    try:
        toggle_NonDutyPaid(driver)
        click_button(driver=driver,
                     id="//a[@id='ctl00_ContentPlaceHolder2_NonDutyPaidItemInfoUc1_lnkItems' and text()='Attach Item']",
                     by=By.XPATH)
        pop_up_492 = process_gd_number_pop_up_492(driver, data)
        if pop_up_492:
            click_button(driver=driver, id="//input[@id='ctl00_ContentPlaceHolder2_btnSaveBottom']", by=By.XPATH)

            WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_btnSaveTop"))
            )
            print(f"Element in pop up Added.")
        else:
            print(f"HS Code Not Found for {data.get('B/E No')}")
            # Cancel the present filling
            click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
    except:
        print(f"Got Error for {data.get('B/E No')}")
        # Cancel the present filling
        click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_pnlTitle"))
        )

def add_excel_data_957(driver: webdriver.Chrome, data, analysis_number, is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
    toggle_NonDutyPaid(driver)
    click_button(driver=driver,
                 id="//a[@id='ctl00_ContentPlaceHolder2_NonDutyPaidItemInfoUc1_lnkItems' and text()='Attach Item']",
                 by=By.XPATH)
    try:
        hs_code = process_gd_number_pop_up_957(driver, data, is_hscode_wise)
        if hs_code:
            process_analysis_number_pop_up_957(driver, analysis_number=analysis_number, hs_code=hs_code)
            click_button(driver=driver, id="//input[@id='ctl00_ContentPlaceHolder2_btnSaveBottom']", by=By.XPATH)

            WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_pnlTitle"))
            )
            print(f"Element in pop up Added.")
        else:
            print(f"HS Code Not Found for {data.get(be_no)}")
            # Cancel the present filling
            click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
    except Exception as e:
        print(f"Got Error for {data.get(be_no)} => {str(e)}")
        # Cancel the present filling
        click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_pnlTitle"))
        )

def process_492(driver,data):
    for idx,obj in enumerate(data):
        if obj.get('B/E No'):
            add_excel_data_492(driver,data=obj)
def process_957(driver,data,analysis_number,is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
    for obj in data:
        if categorize_invoice(obj.get(be_no)) == 'non_local':
            add_excel_data_957(driver,data=obj,analysis_number=analysis_number,is_hscode_wise=is_hscode_wise)
        else:
            print(f"Local Invoice {obj.get(be_no)}")
            add_excel_data_local(driver,data=obj,analysis_number=analysis_number,is_hscode_wise=is_hscode_wise)
def process_localy_purchased_analysis_no_pop_up_957(driver : webdriver.Chrome,analysis_number,hs_code):
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryUc1_btnAnalysisLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",analysis_number,pop_up=True)
    write_text(driver, "txtInputHSCode",hs_code,pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(3)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookupExport_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    wait_for_page_load(driver)
    return True
def process_localy_purchased_pop_up_957(driver : webdriver.Chrome,data,is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryUc1_btnLocalPurchaseLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",data.get(be_no),pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(5)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookup_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    # Locate the table
    
    table = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryUc1_dgItems"))
        )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    row = rows[1]
    cells = row.find_elements(By.TAG_NAME, "td")
    if cells:

        select_link = cells[0].find_element(By.TAG_NAME, "a")
        print(f"No matching row found in the table for PER UNIT VALUE {data.get('PER UNIT VALUE')} and {data.get(be_no)} Selecting 1st row")
        if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
            select_link.click()
            time.sleep(2)
        else:
            print("Select Link is not enabled")
            return None
    else:
        return None
    hs_code = extract_text(driver, "ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryUc1_txtHsCode")
    return hs_code

def add_excel_data_local(driver : webdriver.Chrome,data,analysis_number,is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
    # Wait until the image is present
    toggle_LocalPurchaseItem(driver)
    click_button(driver=driver,id="//a[@id='ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryInfoUc1_lnkItems' and text()='Attach Locally Purchase Item']",by=By.XPATH)
    time.sleep(2)
    
    hs_code = process_localy_purchased_pop_up_957(driver,data,is_hscode_wise)
    if hs_code:
        res_957 = process_localy_purchased_analysis_no_pop_up_957(driver,analysis_number=analysis_number,hs_code=hs_code)
        if res_957:
            click_button(driver=driver,id="//input[@id='ctl00_ContentPlaceHolder2_btnSaveBottom']",by=By.XPATH)
            time.sleep(1)
        else:
            # Cancel the present filling
            click_button(driver=driver,id="ctl00_ContentPlaceHolder2_btnCancelBottom")
            time.sleep(1)
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryInfoUc1_lblTitle"))
        )
        print(f"Element in pop up Added.")
    else:
        print(f"HS Code Not Found for {data.get(be_no)}")
        # Cancel the present filling
        click_button(driver=driver,id="ctl00_ContentPlaceHolder2_btnCancelBottom")

def Non_Duty_Paid_Info(driver,csv_obj:CSVDataExtractor,hs_code,elem_index):
    select_added_item(driver)
    # Wait until the image is present
    
    data_492 = csv_obj.table492_data
    data_957 = csv_obj.table957_data

    process_492(driver,data_492)
    analysis_number = csv_obj.get_analysis_number(hs_code)
    process_957(driver,data_957,analysis_number)
    time.sleep(5)
    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_btnSaveBottom")
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_BasicInfoUc1_pnlTitle"))
    )

def Non_Duty_Paid_Info_multi_po(driver,csv_obj:CSVDataExtractor,hs_code,elem_index):
    select_added_item(driver)
    toggle_NonDutyPaid(driver)
    
    data_957_hs_code_wise = csv_obj.hs_code_wise_tables.get(hs_code)
    data_957 = data_957_hs_code_wise.get('sub_table')

    analysis_number = csv_obj.get_analysis_number(hs_code)
    process_957(driver,data_957,analysis_number,is_hscode_wise=True)
    time.sleep(5)
    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_btnSaveBottom")
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_BasicInfoUc1_pnlTitle"))
    )

def process_multi_single(driver,items_data,prev_idx=0):
    for idx,item_data in enumerate(items_data):
        hs_code = add_item(driver, 1,data=item_data)
        Non_Duty_Paid_Info(driver,item_data.get('csv_obj'),hs_code,elem_index=idx+1+prev_idx)
        print(f"GD Completed For Item : {item_data}")
    return idx
def process_multi_po(driver ,po_obj:MultiPOParse):
    po_tables = po_obj.extracted_data.get('po_tables')
    invoice_nos = po_tables.keys()
    print(invoice_nos)
    for invoice in invoice_nos:
        items_data = po_tables[invoice].get('po_numbers')
        print(items_data)
        for idx,item_data in enumerate(items_data):
            
            data_to_send = po_obj.get_item_info(int(item_data))
            totals = po_tables[invoice].get('totals')
            csv_obj = po_tables[invoice].get('csv_obj')
            data_to_send['csv_obj'] = csv_obj
            data_to_send.update(totals)
            print(data_to_send)
            if not data_to_send.get('hs_code'):
                continue
            else:
                add_item(driver, 1,data=data_to_send)
                
                if idx == 0:
                    Non_Duty_Paid_Info(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                    print(f"Adding HS Code Wise Tables for hs code {data_to_send.get('hs_code')}")
                    Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                    print(f"GD Completed For Item : {item_data}")
                else:
                    Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
            
    return idx

def main(data):
    try:
        print('hi')
        finalStatus = False
        finalMessage =''
        driver = setup_driver()
        transaction_id = data.get("transaction_id")
        username = data["UserName"]
        passwd = data["Password"]
        url = data["URL"]
        login_form_status = True
        login_form_status, login_form_error = populate_username_passowrd_in_login_form(driver, transaction_id,url, username, passwd)
        if login_form_status:
            # pickle.dump(driver.get_cookies(), open("cookies.pkl", "wb"))
            print(f"For transaction_id { transaction_id } Cookies Done...")
            wait = WebDriverWait(driver, 60)
            # Define a condition to wait for page load (you can customize this)
            page_load_condition = EC.presence_of_element_located((By.ID, "sidebar-container"))
            # Wait until the page is fully loaded
            wait.until(page_load_condition)
            time.sleep(3)
            GD_status, GD_error = select_GDS(driver, transaction_id)
            print(f"GD_status: {GD_status}, GD_error: {GD_error}")
            if GD_status:
                fill_form_status, fill_form_error = fill_form(driver, transaction_id,data=data)
                time.sleep(5)
                print(f"Uploading the documents")
                upload_documents(driver,pdf_path=data.get('pdf_paths'))
                time.sleep(5)
                fty_data = data.get('fty_data')
                po_obj = data.get('po_obj')
                items_data = fty_data.get('extracted_data')
                prev_idx = 0
                # prev_idx = process_multi_po(driver,po_obj)
                process_multi_single(driver,items_data,prev_idx)
        else:
            finalMessage = login_form_error

    except Exception as e:
        print(f"For transaction_id { transaction_id } Exception occurred: {e}")
        finalStatus = False
        finalMessage = str(e)
    finally:
        return driver
pl_pdf_path, fty_pdf_path, csv_path = extract_files_club_single()
pl_po_pdf_path, fty_po_pdf_path, csv_po_path,desc_po_path = extract_files_club_po()

fty_parser = MultiSingleParse(fty_pdf_path,csv_path=csv_path)
pl_parser = PlParse(pl_po_pdf_path)
po_parser = MultiPOParse(path=fty_po_pdf_path,csv_path=csv_po_path,des_path=desc_po_path)
pdf_paths = [pl_pdf_path, fty_pdf_path]
final_table_data = add_data_dictionaries(po_parser.extracted_data['final_table'],fty_parser.extracted_data['final_table'])

data = {
    'transaction_id': 1,
    'user_id': 2,
    'URL': 'https://app.psw.gov.pk/app/',
    'UserName': 'CA-01-2688539',
    'Password': 'Express@3833',
    'pdf_paths': pdf_paths,
    'pl_data':pl_parser.extracted_data,
    'fty_data':fty_parser.extracted_data,
    'po_obj':po_parser,
    'final_table':final_table_data,
}
print(fty_parser.extracted_data['extracted_data'][0].get('csv_obj').table492_data)
print(fty_parser.extracted_data['extracted_data'][0].get('csv_obj').table957_data)
print(fty_parser.extracted_data['extracted_data'][0].get('csv_obj').table1_data)
required_keys = list(data.keys())

driver = main(data)


PL PDF Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle\pl.pdf
FTY PDF Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle\fty.pdf
CSV Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle
PL PDF Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multipos\pl.pdf
FTY PDF Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multipos\fty3.pdf
CSV Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multipos
adidas branded garments. export from export facilitation scheme (efs) 134740069 unisex infants 78% cotton 22% recycled polyesterknit pullover dyed
Women 80/20% Pullovers
Searching for CSV file for invoice number 507944 in c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle...
EFS507944.csv
adidas branded garments. export from export facilitation scheme (efs) 134743455 womens 70% cotton 30% recycle polyster knit t-shirts dyed
Women 80/20% T-Shirts
Searching for CSV file for invoice number 507957 in c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle...
EFS507944.csv
EFS5

In [ ]:
items_data = fty_parser.extracted_data.get('extracted_data')
print(items_data)
def process_gd_number_pop_up_492(driver : webdriver.Chrome,data):
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_btnGDLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",data.get('B/E No'),pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(5)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookup_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    # Locate the table
    table = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_dgItems"))
        )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Iterate through the rows, skipping the header
    for row in rows[1:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            try:
                unit_value = float(cells[4].text)
                puv = float(data.get('PER UNIT VALUE'))
                if math.isclose(unit_value, puv, rel_tol=0.1):
                    select_link = cells[0].find_element(By.TAG_NAME, "a")
                    if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                        select_link.click()
                    else:
                        return None
                    time.sleep(2)
                    break
            except ValueError:
                print(f"Skipping row due to non-numeric value: {cells[4].text}")
    else:
        # select 1st row
        row = rows[1]
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            select_link = cells[0].find_element(By.TAG_NAME, "a")
            print(f"No matching row found in the table for PER UNIT VALUE {data.get('PER UNIT VALUE')} and {data.get('B/E No')} Selecting 1st row")
            if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                select_link.click()
            else:
                print("Select Link is not enabled")
                return None
        else:
            return None
    try:
        Quantity = float(extract_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity"))
    except :
        Quantity = 0.0
    try:
        CONSUMED = float(data.get('NOW CONSUMED'))
    except :
        CONSUMED = 0.0
    if CONSUMED < Quantity:
        write_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity",data.get('Now Consume'),pop_up=True)
    return True

def process_gd_number_pop_up_957(driver : webdriver.Chrome,data,is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_btnGDLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",data.get(be_no),pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(5)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookup_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    # Locate the table
    table = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_dgItems"))
        )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Iterate through the rows, skipping the header
    for row in rows[1:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            print("The celss text is : ",cells[4].text)
            try:
                unit_value = float(cells[4].text)
                puv = float(data.get('PER UNIT VALUE'))
                if math.isclose(unit_value, puv, rel_tol=0.1):
                    select_link = cells[0].find_element(By.TAG_NAME, "a")
                    if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                        select_link.click()
                    else:
                        return None
                    time.sleep(2)
                    break
            except ValueError:
                print(f"Skipping row due to non-numeric value: {cells[4].text}")
    else:
        print('In rows')
        # select 1st row
        row = rows[1]
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            select_link = cells[0].find_element(By.TAG_NAME, "a")
            print(f"No matching row found in the table for PER UNIT VALUE {data.get('PER UNIT VALUE')} and {data.get(be_no)} Selecting 1st row")
            if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                select_link.click()
                time.sleep(5)
            else:
                print("Select Link is not enabled")
                return None
        else:
            return None
    try:
        Quantity = float(extract_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity"))
    except :
        Quantity = 0.0
    try:
        CONSUMED = float(data.get('NOW CONSUMED'))
    except :
        CONSUMED = 0.0
    if CONSUMED < Quantity:
        write_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity",data.get('NOW CONSUMED'),pop_up=True)
    hs_code = extract_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtHsCode")
    return hs_code

def process_analysis_number_pop_up_957(driver : webdriver.Chrome,analysis_number,hs_code):
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_btnAnalysisLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",analysis_number,pop_up=True)
    write_text(driver, "txtInputHSCode",hs_code,pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(3)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookupExport_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    time.sleep(2)
    wait_for_page_load(driver)

def process_multi_single(driver,items_data,prev_idx=0):
    for idx,item_data in enumerate(items_data):
        if idx == 0:
            continue
        else:
            hs_code = add_item(driver, 1,data=item_data)
            Non_Duty_Paid_Info(driver,item_data.get('csv_obj'),hs_code,elem_index=idx+1+prev_idx)
            print(f"GD Completed For Item : {item_data}")
    return idx
process_multi_single(driver,items_data,1)

In [ ]:
def find_hs_code(description):
    description = description.lower().strip().replace('\n', '')
    print(description)
    material_category = determine_category(description)

    garment_type = None
    gender_category = None

    if 't-shirt' in description or 't shirt' in description:
        garment_type = 'T-Shirts'
    elif 'shirt' in description:
        garment_type = 'Shirts'
    elif 'pant' in description or 'shorts' in description:
        garment_type = 'Pants'
    elif 'jacket' in description:
        garment_type = 'Jackets'
    elif 'track suit' in description:
        garment_type = 'Track Suits'
    elif 'pullover' in description:
        garment_type = 'Pullovers'

    if 'unisex' in description:
        gender_category = 'Women'
    elif 'women' in description or 'ladies' in description or 'girl' in description:
        gender_category = 'Women'
    elif 'men' in description or 'gentle' in description or 'boy' in description or 'kid' in description:
        gender_category = 'Men'
    
    print(f"{gender_category} {material_category} {garment_type}")
    if material_category and garment_type and gender_category:
        key = f"{gender_category} {material_category} {garment_type}"
        return hs_codes.get(key)

    return None

def process_multi_po(driver ,po_obj:MultiPOParse):
    po_tables = po_obj.extracted_data.get('po_tables')
    invoice_nos = po_tables.keys()
    print(invoice_nos)
    for invoice in invoice_nos:
        
        items_data = po_tables[invoice].get('po_numbers')
        print(items_data)
        for idx,item_data in enumerate(items_data):
            if idx == 0:
                continue
            else:
                data_to_send = po_obj.get_item_info(int(item_data))
                totals = po_tables[invoice].get('totals')
                csv_obj = po_tables[invoice].get('csv_obj')
                data_to_send['csv_obj'] = csv_obj
                data_to_send.update(totals)
                data_to_send['Carton'] = data_to_send['CTNS']
                data_to_send['Quantity'] = data_to_send['PCS']
                print(data_to_send)
                # if not data_to_send.get('hs_code'):
                #     continue
                # else:
                #     add_item(driver, 1,data=data_to_send)
                    
                #     if idx == 0:
                #         # Non_Duty_Paid_Info(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                #         print(f"Adding HS Code Wise Tables for hs code {data_to_send.get('hs_code')}")
                #         Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                #         print(f"GD Completed For Item : {item_data}")
                #     else:
                #         Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
            
    return idx
process_multi_po(driver=driver,po_obj=po_parser)
po_parser.po_data

In [ ]:
driver = main(data)

In [ ]:
def find_hs_code(description):
    description = description.lower().strip().replace('\n', '')
    print(description)
    material_category = determine_category(description)

    garment_type = None
    gender_category = None

    if 't-shirt' in description or 't shirt' in description:
        garment_type = 'T-Shirts'
    elif 'shirt' in description:
        garment_type = 'Shirts'
    elif 'pant' in description or 'shorts' in description:
        garment_type = 'Pants'
    elif 'jacket' in description:
        garment_type = 'Jackets'
    elif 'track suit' in description:
        garment_type = 'Track Suits'
    elif 'pullover' in description:
        garment_type = 'Pullovers'

    if 'unisex' in description:
        gender_category = 'Women'
    elif 'women' in description or 'ladies' in description or 'girl' in description:
        gender_category = 'Women'
    elif 'men' in description or 'gentle' in description or 'boy' in description or 'kid' in description:
        gender_category = 'Men'
    
    print(f"{gender_category} {material_category} {garment_type}")
    if material_category and garment_type and gender_category:
        key = f"{gender_category} {material_category} {garment_type}"
        return hs_codes.get(key)

    return None

def process_multi_po(driver ,po_obj:MultiPOParse):
    po_tables = po_obj.extracted_data.get('po_tables')
    invoice_nos = po_tables.keys()
    print(invoice_nos)
    for invoice in invoice_nos:
        items_data = po_tables[invoice].get('po_numbers')
        print(items_data)
        for idx,item_data in enumerate(items_data):
            if idx == 0:
                continue
            else:
                data_to_send = po_obj.get_item_info(int(item_data))
                totals = po_tables[invoice].get('totals')
                csv_obj = po_tables[invoice].get('csv_obj')
                data_to_send['csv_obj'] = csv_obj
                data_to_send.update(totals)
                print(data_to_send)
                if not data_to_send.get('hs_code'):
                    continue
                else:
                    add_item(driver, 1,data=data_to_send)
                    
                    if idx == 0:
                        # Non_Duty_Paid_Info(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                        print(f"Adding HS Code Wise Tables for hs code {data_to_send.get('hs_code')}")
                        Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                        print(f"GD Completed For Item : {item_data}")
                    else:
                        Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
            
    return idx
process_multi_po(driver=driver,po_obj=po_parser)

In [ ]:
driver = main(data)

In [ ]:
def find_hs_code(description):
    description = description.lower().strip().replace('\n', '')
    print(description)
    material_category = determine_category(description)

    garment_type = None
    gender_category = None

    if 't-shirt' in description or 't shirt' in description:
        garment_type = 'T-Shirts'
    elif 'shirt' in description:
        garment_type = 'Shirts'
    elif 'pant' in description or 'shorts' in description:
        garment_type = 'Pants'
    elif 'jacket' in description:
        garment_type = 'Jackets'
    elif 'track suit' in description:
        garment_type = 'Track Suits'
    elif 'pullover' in description:
        garment_type = 'Pullovers'

    if 'unisex' in description:
        gender_category = 'Women'
    elif 'women' in description or 'ladies' in description or 'girl' in description:
        gender_category = 'Women'
    elif 'men' in description or 'gentle' in description or 'boy' in description or 'kid' in description:
        gender_category = 'Men'
    
    print(f"{gender_category} {material_category} {garment_type}")
    if material_category and garment_type and gender_category:
        key = f"{gender_category} {material_category} {garment_type}"
        return hs_codes.get(key)

    return None

def process_multi_po(driver ,po_obj:MultiPOParse):
    po_tables = po_obj.extracted_data.get('po_tables')
    invoice_nos = po_tables.keys()
    print(invoice_nos)
    for invoice in invoice_nos:
        items_data = po_tables[invoice].get('po_numbers')
        print(items_data)
        for idx,item_data in enumerate(items_data):
            if idx == 0:
                continue
            else:
                data_to_send = po_obj.get_item_info(int(item_data))
                totals = po_tables[invoice].get('totals')
                csv_obj = po_tables[invoice].get('csv_obj')
                data_to_send['csv_obj'] = csv_obj
                data_to_send.update(totals)
                print(data_to_send)
                if not data_to_send.get('hs_code'):
                    continue
                else:
                    add_item(driver, 1,data=data_to_send)
                    
                    if idx == 0:
                        # Non_Duty_Paid_Info(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                        print(f"Adding HS Code Wise Tables for hs code {data_to_send.get('hs_code')}")
                        Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                        print(f"GD Completed For Item : {item_data}")
                    else:
                        Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
            
    return idx
process_multi_po(driver=driver,po_obj=po_parser)

In [4]:
driver = main(data)

hi
For transaction_id 1 Cookies Done...
For transaction_id 1 Selecting GDS...
Found traders_button.
iframe Switched.
Found company_element.
Switched to default_content.
Found goods_declaration.
Found Create_Export_GD.
Found ctl00_ContentPlaceHolder2_GdSelectionExport1_ddlConsignmentType dropdown_elem.
Found ctl00_ContentPlaceHolder2_GdSelectionExport1_ddlDeclarationType dropdown_elem.
Found Create_button.
Clicked Create_button.
GD_status: True, GD_error: 
For transaction_id 1 Filling the Form...
Found ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlCollectorate dropdown_elem.
Found ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlModeOfTransport dropdown_elem.
Found ctl00_ContentPlaceHolder2_ConsigneeInfoUc1_txtConsignorName element.
Found ctl00_ContentPlaceHolder2_ConsigneeInfoUc1_txtConsignorAddress element.
Found ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlConsignmentMode dropdown_elem.
Found ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtBlNo element.
Found ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtBlDat

In [8]:
def process_duty_drawback(driver,description):
    table = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_pnldutydrawback"))
        )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Iterate through the rows, skipping the header
    for row in rows[2:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if 'dye' in description.lower() and 'dyed or printed' in cells[1].text.lower():
            cells[0].click()
            return True
        elif 'bleach' in description.lower() and 'bleached blended' in cells[1].text.lower():
            cells[0].click()
            return True
        elif '100% polyester' in description.lower() and 'grey blended' in cells[1].text.lower():
            cells[2].click()
            return True
    else:
        row = rows[2]
        cells = row.find_elements(By.TAG_NAME, "td")
        cells[0].click()
        return True
process_duty_drawback(driver,'bleach Unisex Infants 78% Cotton 22% Recycled Polyester')


True

In [2]:
def find_hs_code(description):
    description = description.lower().strip().replace('\n', '')
    print(description)
    material_category = determine_category(description)

    garment_type = None
    gender_category = None

    if 't-shirt' in description or 't shirt' in description:
        garment_type = 'T-Shirts'
    elif 'shirt' in description:
        garment_type = 'Shirts'
    elif 'pant' in description or 'shorts' in description:
        garment_type = 'Pants'
    elif 'jacket' in description:
        garment_type = 'Jackets'
    elif 'track suit' in description:
        garment_type = 'Track Suits'
    elif 'pullover' in description:
        garment_type = 'Pullovers'

    if 'unisex' in description:
        gender_category = 'Women'
    elif 'women' in description or 'ladies' in description or 'girl' in description:
        gender_category = 'Women'
    elif 'men' in description or 'gentle' in description or 'boy' in description or 'kid' in description:
        gender_category = 'Men'
    
    print(f"{gender_category} {material_category} {garment_type}")
    if material_category and garment_type and gender_category:
        key = f"{gender_category} {material_category} {garment_type}"
        return hs_codes.get(key)

    return None

def process_multi_po(driver ,po_obj:MultiPOParse):
    po_tables = po_obj.extracted_data.get('po_tables')
    invoice_nos = po_tables.keys()
    print(invoice_nos)
    for invoice in invoice_nos:
        items_data = po_tables[invoice].get('po_numbers')
        print(items_data)
        for idx,item_data in enumerate(items_data):
            if idx == 0:
                continue
            else:
                data_to_send = po_obj.get_item_info(int(item_data))
                totals = po_tables[invoice].get('totals')
                csv_obj = po_tables[invoice].get('csv_obj')
                data_to_send['csv_obj'] = csv_obj
                data_to_send.update(totals)
                print(data_to_send)
                if not data_to_send.get('hs_code'):
                    continue
                else:
                    add_item(driver, 1,data=data_to_send)
                    
                    if idx == 0:
                        # Non_Duty_Paid_Info(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                        print(f"Adding HS Code Wise Tables for hs code {data_to_send.get('hs_code')}")
                        Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                        print(f"GD Completed For Item : {item_data}")
                    else:
                        Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
            
    return idx
process_multi_po(driver=driver,po_obj=po_parser)

dict_keys(['508645', '520064'])
['0134793136', '0135021120', '0135037910', '0135038956', '0135095361', '0135151371', '0135153369', '0135157520', '0135201839', '0135201858', '0135239107', '0135253471']
mens 100% cotton knit pants dyed
Men 100% Pants
{'CTNS': 4, 'description': 'Mens 100% Cotton Knit Pants Dyed', 'hs_code': '6103.4200', 'csv_obj': <helpers.excelhelper.CSVDataExtractor object at 0x0B453088>, 'Quantity': 6847.0, 'Carton': 386.0, 'Gross Weight': 2470.0, 'Net Weight': 2079.0, 'Net Net Weight': 1970.0, 'PO Net Amount': 41535.39, 'VAT': 0.0, 'invoice_number': '508645'}
For transaction_id 1 Adding Item...
Exception: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.89)
Stacktrace:
	GetHandleVerifier [0x00007FF74E909632+30946]
	(No symbol) [0x00007FF74E8BE3C9]
	(No symbol) [0x00007FF74E7B6FDA]
	(No symbol) [0x00007FF74E78CB85]
	(No symbol) [0x00007FF74E8337A7]
	(No symbol) [0x00007FF74E84A771]
	(No symb

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.89)
Stacktrace:
	GetHandleVerifier [0x00007FF74E909632+30946]
	(No symbol) [0x00007FF74E8BE3C9]
	(No symbol) [0x00007FF74E7B6FDA]
	(No symbol) [0x00007FF74E78CB85]
	(No symbol) [0x00007FF74E8337A7]
	(No symbol) [0x00007FF74E84A771]
	(No symbol) [0x00007FF74E82C813]
	(No symbol) [0x00007FF74E7FA6E5]
	(No symbol) [0x00007FF74E7FB021]
	GetHandleVerifier [0x00007FF74EA3F83D+1301229]
	GetHandleVerifier [0x00007FF74EA4BDB7+1351783]
	GetHandleVerifier [0x00007FF74EA42A03+1313971]
	GetHandleVerifier [0x00007FF74E93DD06+245686]
	(No symbol) [0x00007FF74E8C758F]
	(No symbol) [0x00007FF74E8C3804]
	(No symbol) [0x00007FF74E8C3992]
	(No symbol) [0x00007FF74E8BA3EF]
	BaseThreadInitThunk [0x00007FFB17847374+20]
	RtlUserThreadStart [0x00007FFB17B1CC91+33]
